## Let's see if we can wire this up to an openbb function
(Even if it's hardcoded for now)


- Agent
- Tools
- Prompt
- AgentExecutor (runtime)


In [1]:
import inspect

from openbb import obb
from openbb_core.app.router import CommandMap
from openbb_core.app.provider_interface import ProviderInterface

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool, Tool, AgentExecutor, AgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools import StructuredTool

In [3]:
from pydantic.v1 import create_model, BaseModel
from pydantic.v1.fields import FieldInfo
from pydantic_core import PydanticUndefinedType

Goal: let's make it easy to convert an OpenBB function --> Langchain / openai function.

Schema comes from the ProviderInterface  
Function / Endpoint map comes from CommandMap + scraping the Members

In [4]:
def _fetch_obb_module(openbb_command_root):
    module_path_split = openbb_command_root.split("/")[1:]
    module_path = '.'.join(module_path_split)

    # Iteratively get module
    module = obb
    for attr in module_path.split('.'):
        module=getattr(module, attr)

    return module

def _fetch_schemas(openbb_command_root):
    # Ugly hack to make it compatiable
    # (even though we convert it back)
    # so that we have a nicer API.
    module_root_path = openbb_command_root.replace("/", ".")
    schemas = {
        k.replace('.', '/'): v for k, v in obb.coverage.command_model.items() if module_root_path in k
    }
    return schemas

def _fetch_callables(openbb_command_root):
    module = _fetch_obb_module(openbb_command_root)
    members = inspect.getmembers(module)
    members_dict = {x[0]: x[1] for x in members if '__' not in x[0] and '_run' not in x[0]}

    schemas = _fetch_schemas(openbb_command_root)

    # Create callables dict, with the same key as used in the schemas
    callables = {}
    for k in schemas.keys():
        try:
            callables[k] = members_dict[k.split('/')[-1]] 
        except KeyError:  # Sometimes we don't have a specific callable for an endpoint, so we skip.
            pass
    return callables

fundamental_schemas = _fetch_schemas('/equity/fundamental')
fundamental_callables = _fetch_callables('/equity/fundamental')

In [5]:
from typing import get_args
def _fetch_outputs(schema):
    outputs = []
    output_fields = schema['openbb']['Data']['fields']
    for name, t in output_fields.items():
        if isinstance(t.annotation, type):
            type_str = t.annotation.__name__
        else:
            type_str = str(t.annotation).replace("typing.", "")
        outputs.append((name, type_str))
    return outputs

In [6]:
from typing import get_origin, Literal

def from_schema_to_pydantic_model(model_name, schema):
    create_model_kwargs = {}
    for field, field_info in schema.items():

        field_type = field_info.annotation

        # Handle default values
        if not isinstance(field_info.default, PydanticUndefinedType):
            field_default_value = field_info.default
            new_field_info = FieldInfo(  # Weird hack, because of how the default field value works
                description=field_info.description,
                default=field_default_value,
            )
        else:
            new_field_info = FieldInfo(
                description=field_info.description,
            )
        create_model_kwargs[field] = (field_type, new_field_info)
    return create_model(model_name, **create_model_kwargs)

In [7]:

def return_results(func):
    """Return the results rather than the OBBject."""
    def wrapper_func(*args, **kwargs):
        return func(*args, **kwargs).results
    return wrapper_func


In [8]:
def from_openbb_to_langchain_func(openbb_callable, openbb_schema):
    func_name = openbb_callable.__name__
    func_schema = openbb_schema['openbb']['QueryParams']['fields']
    
    pydantic_model = from_schema_to_pydantic_model(
        model_name=f'{func_name}InputModel',
        schema=func_schema
    )

    outputs = _fetch_outputs(openbb_schema)
    description = openbb_callable.__doc__.split("\n")[0]
    description += "\nThe following data is available:\n\n"
    description += "\n".join(e[0] for e in outputs)

    tool = StructuredTool(
        name = func_name,
        func=return_results(openbb_callable),
        description=description,
        args_schema=pydantic_model
    )

    return tool

In [9]:
def map_openbb_functions_to_langchain_tools(schemas_dict, callables_dict):
    tools = []
    for route in callables_dict.keys():
        tool = from_openbb_to_langchain_func(
            openbb_callable=callables_dict[route],
            openbb_schema=schemas_dict[route]
        )
        tools.append(tool)
    return tools

In [10]:
def map_openbb_collection_to_langchain_tools(openbb_command_root):
    schemas = _fetch_schemas(openbb_command_root)
    callables = _fetch_callables(openbb_command_root)
    tools = map_openbb_functions_to_langchain_tools(
        schemas_dict=schemas,
        callables_dict=callables
    )
    return tools

In [11]:
fundamental_tools = map_openbb_collection_to_langchain_tools("/equity/fundamental")
compare_tools = map_openbb_collection_to_langchain_tools("/equity/compare")

In [12]:
len(compare_tools)

1

In [181]:
# Let's add some memory
MEMORY_KEY = "chat_history"

# Create our prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You're a powerful financial advisor.
            Do your best to answer the questions,
            and make use of the tools provided, paying special attention to the outputs they provide."""
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        (
            "user",
            "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

# Bind our tools
llm = ChatOpenAI(model="gpt-4-1106-preview")
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in compare_tools])  # Never forget!


# Create the agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=compare_tools, verbose=True)

In [183]:
type(agent)

langchain.schema.runnable.base.RunnableSequence

In [156]:
OpenAIFunctionsAgentOutputParser?

Init signature: OpenAIFunctionsAgentOutputParser() -> None
Docstring:     
Parses a message into agent action/finish.

Is meant to be used with OpenAI models, as it relies on the specific
function_call parameter from OpenAI to convey what tools to use.

If a function_call parameter is passed, then that is used to get
the tool and tool input.

If one is not passed, then the AIMessage is assumed to be the final output.
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/miniforge3/envs/ds/lib/python3.10/site-packages/langchain/agents/output_parsers/openai_functions.py
Type:           ModelMetaclass
Subclasses:     

In [ ]:
chat_history = []

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

human_input = "List the competitors for TSLA. Use tools."

response = agent_executor.invoke(
    {
        "input": human_input,
        "chat_history": chat_history
    }
    
)

chat_history.extend(
    [
        HumanMessage(content=human_input),
        AIMessage(content=response["output"])
    ]
)



> Entering new AgentExecutor chain...

Invoking: `peers` with `{'symbol': 'TSLA'}`


peers_list=['XPEV', 'LI', 'RIVN', 'LCID', 'GM', 'NIO', 'F', 'FSR', 'MULN']The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's incre

In [ ]:
print(response['output'])

The competitors for Tesla, Inc. (TSLA) include various companies in the electric vehicle (EV) and broader automotive industry. Here's a list of some of the competitors:

1. XPeng Inc. (XPEV) - A Chinese electric vehicle manufacturer.
2. Li Auto Inc. (LI) - Another Chinese company that specializes in electric vehicles.
3. Rivian Automotive, Inc. (RIVN) - An American electric vehicle automaker that focuses on trucks and SUVs.
4. Lucid Group, Inc. (LCID) - An American EV manufacturer that produces luxury electric vehicles.
5. General Motors Company (GM) - A traditional automotive company based in the United States that is expanding into the EV market.
6. NIO Inc. (NIO) - A Chinese electric vehicle manufacturer that produces smart, connected EVs.
7. Ford Motor Company (F) - An established global automotive manufacturer that's increasing its investment in electric vehicles.
8. Fisker Inc. (FSR) - An American electric vehicle company that is developing electric passenger vehicles.
9. Mullen 

# Let's explore different agents

In [ ]:
from langchain import hub
from langchain.agents import initialize_agent
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

# Let's partially fill out the prompt
tools = compare_tools
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools])
)

llm_with_stop = llm.bind(stop=["\nObservation"])

In [ ]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [ ]:
response = agent_executor.invoke(
    {
        "input": (
            "Who are TSLA's competitors? "
            "For each competitor, find out who their competitor is, "
            "and repeat this one more time. "
            "Then write a python script that will generate a plot "
            "showing the network of relationships."
        )
    }
)




> Entering new AgentExecutor chain...
Thought: To answer the question, I first need to find out who Tesla's competitors are. Then, for each of those competitors, I need to find out who their competitors are, creating a network of relationships. I will use the Equity Peers tool to find this information.

Action: peers
Action Input: TSLApeers_list=['XPEV', 'LI', 'RIVN', 'LCID', 'GM', 'NIO', 'F', 'FSR', 'MULN']I have the list of Tesla's competitors. Now, I need to find out the competitors for each of these companies listed in peers_list to create a network of relationships. I will do this by using the Equity Peers tool for each of Tesla's competitors.

Action: peers
Action Input: XPEVpeers_list=['TSLA', 'LI', 'RIVN', 'LCID', 'NIO', 'BYDDY', 'F', 'FSR', 'MULN']Now I have the competitors for XPEV. I will need to repeat this process for each of Tesla's competitors to build the network of relationships.

Action: peers
Action Input: LIpeers_list=['NIO', 'RIVN', 'LCID', 'TSLA', 'XPEV', 'FSR',

## Let's try and do tool retrieval

In [75]:
from langchain.agents import AgentExecutor, AgentOutputParser, LLMSingleActionAgent, Tool

from langchain.chains import LLMChain
from langchain.llms import OpenAI, OpenAIChat
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish

In [17]:
# We embed the tools using a vector store.
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS

In [23]:
fundamental_tool_docs = [
    Document(page_content=t.description, metadata={"index": i}) for i, t in enumerate(fundamental_tools)
]

In [26]:
vector_store = FAISS.from_documents(fundamental_tool_docs, OpenAIEmbeddings())

In [50]:
retriever = vector_store.as_retriever()

def get_tools(query):
    docs = retriever.get_relevant_documents(query)
    return [fundamental_tools[d.metadata["index"]] for d in docs]
    return docs

In [102]:
# We have the tool retriever. Now we need to pair it with our agent.
# Let's start with the prompt

template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to give your answers in a bulleted list.

Question: {input}
{agent_scratchpad}""" 

In [187]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant"
        ),
        (
            "human", "{input}",
        ),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)


ValueError: Expected 2-tuple of (role, template), got ('human', '{input}', MessagesPlaceholder(variable_name='agent_scratchpad'))

In [153]:
agent.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage, langchain.schema.messages.ToolMessage]]}, messages=[SystemMessage(content='Do your best to answer the questions. Feel free to use any tools available to look up relevant information, only if necessary'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [169]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    input_variables=["input", "intermediate_steps"]
)

In [170]:
from typing import Union
import re
import json

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        try:
            action_input = json.loads(action_input)
        except json.JSONDecodeError:
            pass

        print(action_input)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [171]:
output_parser = OpenAIFunctionsAgentOutputParser()
output_parser

OpenAIFunctionsAgentOutputParser()

In [172]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-1106-preview")

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"])
    }
    | llm
    | OpenAIFunctionsAgentOutputParser()

)

In [173]:
query = "What's the income for TSLA?"
tools = get_tools(query)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [175]:
llm_chain

LLMChain(prompt=CustomPromptTemplate(input_variables=['input', 'intermediate_steps'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to give your answers in a bulleted list.\n\nQuestion: {input}\n{agent_scratchpad}', tools_getter=<function get_tools at 0x2976aab90>), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x2b90e7ac0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x2b90efca0>, model_name='gpt-4-1106-preview

In [179]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [180]:
agent_executor.invoke({
    "input": "What is the income for TSLA in 2022?",
})



> Entering new AgentExecutor chain...


ValueError: Can only parse messages